## Mental Health FAQ Chatbot
This notebook builds a knowledge-based chatbot that answers questions from the Mental Health FAQ dataset.

In [ ]:
"""
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = ""

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "narendrageek/mental-health-faq-for-chatbot",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())
"""

1. Data Loading and Exploration

In [2]:
import pandas as pd
# Load the dataset (assuming CSV is in same directory)
df = pd.read_csv('Mental_Health_FAQ.csv')
# Display first few rows
print(f"Dataset shape: {df.shape}")
df.head()

Dataset shape: (98, 3)


,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."


2. Data Processing

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [4]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\markn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\markn\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\markn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
def preprocess_text(text):
    """Preprocess text for NLP tasks"""
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize
    tokens = nltk.word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

In [6]:
# Create processed text column
df['processed'] = df['Questions'].apply(preprocess_text) + " " + df['Answers'].apply(preprocess_text)
df.sample(3)

,Question_ID,Questions,Answers,processed
77,6062232,What's the difference between sadness and depr...,"Sadness is a normal reaction to a loss, disapp...",whats difference sadness depression sadness no...
23,8927672,Where can I learn more information about creat...,Similar to a medical advance directive or a he...,learn information creating psychiatric advance...
73,7995219,What's the difference between mental health an...,‘Mental health’ and ‘mental illness’ are incre...,whats difference mental health mental illness ...


4. Vectorization and Similarity Matching

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
# Initialize and fit TF-IDF vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['processed'])

5. Chatbot Implementation

In [9]:
def get_most_similar_response(user_input):
    """Find most similar FAQ answer to user input"""
    # Preprocess user input
    processed_input = preprocess_text(user_input)
    
    # Vectorize input
    input_vector = vectorizer.transform([processed_input])
    
    # Calculate similarities
    similarities = cosine_similarity(input_vector, tfidf_matrix)
    
    # Get index of most similar FAQ entry
    max_sim_idx = similarities.argmax()
    max_similarity = similarities[0, max_sim_idx]
    
    # Return answer with confidence
    if max_similarity < 0.2:
        return "I'm not sure I understand. Could you please rephrase your question?", 0
    
    return df.iloc[max_sim_idx]['Answers'], max_similarity

6. Chatbot Interface

In [10]:
def run_chatbot():
    print("\nMental Health FAQ Chatbot")
    print("Type 'quit' to exit\n")
    print("How can I help you today?")
    
    while True:
        user_input = input("\nYou: ")
        
        if user_input.lower() == 'quit':
            print("Bot: Thank you for chatting. Take care!")
            break
            
        response, confidence = get_most_similar_response(user_input)
        
        if confidence < 0.2:
            print(f"Bot: {response}")
        else:
            print(f"Bot: {response}")
            print(f"[Confidence: {confidence:.2f}]")

7. Test the Chatbot

In [ ]:
# Run the chatbot interface
run_chatbot()